## Data cleaning

In [135]:
# library imports

import numpy as np
import pandas as pd
import csv
import datetime as dt
import statistics
import time
from ast import literal_eval
import itertools
import re
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## SteamSpy data

In [4]:
raw_steamspy_data = pd.read_csv('/Users/sstefanovic/Documents/GitHub/LHL-Capstone_Project/data/steamspy_data_raw.csv')
raw_steamspy_data.head()

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
0,60,Ricochet,Valve,Valve,NaN,4218,929,0,"5,000,000 .. 10,000,000",24,0,3,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,3,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110..."
1,20,Team Fortress Classic,Valve,Valve,NaN,6357,979,0,"5,000,000 .. 10,000,000",136,0,20,0,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,75,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271..."
2,10,Counter-Strike,Valve,Valve,NaN,214651,5470,0,"10,000,000 .. 20,000,000",7720,134,232,147,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,10895,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3..."
3,70,Half-Life,Valve,Valve,NaN,85453,3070,0,"2,000,000 .. 5,000,000",391,109,168,184,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,525,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ..."
4,80,Counter-Strike: Condition Zero,Valve,Valve,NaN,20911,1995,0,"5,000,000 .. 10,000,000",617,1,71,1,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,232,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ..."


In [21]:
raw_steamspy_data.shape

(61164, 20)

In [22]:
raw_steamspy_data.dtypes

appid                int64
name                object
developer           object
publisher           object
score_rank         float64
positive             int64
negative             int64
userscore            int64
owners              object
average_forever      int64
average_2weeks       int64
median_forever       int64
median_2weeks        int64
price              float64
initialprice       float64
discount           float64
languages           object
genre               object
ccu                  int64
tags                object
dtype: object

In [5]:
#checking Null values in all columns
raw_steamspy_data.isnull().sum()

appid                  0
name                   9
developer            229
publisher            152
score_rank         61121
positive               0
negative               0
userscore              0
owners                 0
average_forever        0
average_2weeks         0
median_forever         0
median_2weeks          0
price                 29
initialprice          22
discount              22
languages             84
genre                289
ccu                    0
tags                   0
dtype: int64

In [7]:
#name column
raw_steamspy_data[raw_steamspy_data['name'].isnull()]

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
476,17760,NaN,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
10647,506630,NaN,NaN,Big Fish Games,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Action': 21, 'Adventure': 21}"
13236,576960,NaN,Nuclear Tales,NaN,NaN,110,26,0,"20,000 .. 50,000",0,0,0,0,299.0,1499.0,80.0,"English, Spanish - Spain, French, German","Adventure, Casual, Indie, RPG, Simulation, Str...",0,"{'Adventure': 377, 'Choose Your Own Adventure'..."
21544,806220,NaN,"Jochen Heizmann, Emmanuel Henné, Intermediaware",Asylum Square,NaN,26,13,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,English,"Action, Indie",0,"{'Side Scroller': 102, ""Shoot 'Em Up"": 97, 'Bu..."
27188,965340,NaN,2nd Studio,2nd Studio,NaN,55,7,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,"English, German, Danish, Japanese, Russian, Si...","Action, Indie, Simulation",0,"{'Indie': 34, 'Action': 31, 'Sexual Content': ..."
30890,1071920,NaN,Ultimo Games,Ultimo Games,NaN,19,0,0,"0 .. 20,000",0,0,0,0,399.0,399.0,0.0,English,"Action, Indie",0,"{'Indie': 33, 'Action': 33, 'Retro': 16, 'Arca..."
32396,1116910,NaN,NaN,NaN,NaN,12,16,0,"0 .. 20,000",0,0,0,0,699.0,699.0,0.0,NaN,"Action, Adventure, Casual, Indie, RPG, Simulat...",0,"{'Casual': 33, 'Adventure': 31, 'Indie': 30, '..."
35742,1216770,NaN,NaN,Driven Arts,NaN,1,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, French",NaN,0,"{'Casual': 104, 'Indie': 98, 'RPGMaker': 94, '..."
52840,1860160,NaN,NaN,Mens Sana Interactive,NaN,2,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,English,NaN,0,"{'FPS': 286, 'PvE': 279, 'Shooter': 275, 'Acti..."


In [9]:
#name column
raw_steamspy_data[raw_steamspy_data['name'] == 'none']

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4839,339860,none,NaN,Green Sauce Games,NaN,84,37,0,"50,000 .. 100,000",322,0,324,0,0.0,0.0,0.0,English,NaN,0,"{'Indie': 53, 'RPG': 33, 'Adventure': 32, 'Mas..."
6620,385020,none,none,Valsar,NaN,13,33,0,"20,000 .. 50,000",261,0,263,0,0.0,0.0,0.0,"English, French, Italian, German, Spanish - Sp...","Casual, Indie, Massively Multiplayer, Strategy",1,"{'Indie': 47, 'Strategy': 31, 'Casual': 21, 'M..."
7140,398970,none,none,none,NaN,8,23,0,"50,000 .. 100,000",255,0,261,0,0.0,0.0,0.0,English,"Adventure, Indie, Massively Multiplayer, RPG, ...",0,"{'Massively Multiplayer': 38, 'Strategy': 23, ..."


In [12]:
#developer
raw_steamspy_data[raw_steamspy_data['developer'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
14052,600600,MakeThatMoney,NaN,My Way Games,NaN,21,25,0,"0 .. 20,000",242,0,242,0,99.0,99.0,0.0,English,NaN,0,"{'Simulation': 67, 'Casual': 61, 'Top-Down': 4..."
31851,1100510,Demo,NaN,nightowls,NaN,101,41,0,"20,000 .. 50,000",0,0,0,0,0.0,0.0,0.0,English,NaN,0,"{'Free to Play': 37, 'Action': 34, 'Violent': ..."
8500,439660,Tower Unite Dedicated Server,NaN,MLB.com,NaN,3,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Free to Play': 40, 'Mini Golf': 8}"
25328,910430,The War of the Worlds: Andromeda,NaN,Mediamas Europa,NaN,5,11,0,"0 .. 20,000",0,0,0,0,99.0,99.0,0.0,"English, Spanish - Spain","Action, Indie",0,"{'Action': 21, 'Indie': 21}"
14556,615490,I'm Awesome,NaN,Team D-13,NaN,13,8,0,"0 .. 20,000",0,0,0,0,199.0,199.0,0.0,English,NaN,0,"{'Action': 35, 'Indie': 33, 'Casual': 33, 'Vio..."


In [15]:
#publisher
raw_steamspy_data[raw_steamspy_data['publisher'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
16995,684740,Dungeons 3 Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
2107,239490,America's Army: Proving Grounds Beta (Closed),NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,0.0,0.0,NaN,NaN,1,[]
6146,371510,Gunnheim,SIEIDI Ltd,NaN,NaN,128,191,0,"50,000 .. 100,000",161,0,243,0,999.0,999.0,0.0,English,"Action, Indie",0,"{'Action': 69, 'Indie': 67, 'Co-op': 42, 'Twin..."
1228,72530,Arcadia Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
17933,709970,SwapQuest Demo,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]


In [13]:
#price
raw_steamspy_data[raw_steamspy_data['price'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
4221,321210,Blade Symphony Beta,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
17582,700580,Rust - Staging Branch,NaN,NaN,NaN,0,0,0,"0 .. 20,000",266,0,3,0,NaN,NaN,NaN,NaN,NaN,5,[]
21811,813350,Ben 10 VR,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
15046,630790,PSYCHO-PASS,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,1,[]
11580,530940,BIOHAZARD 7 resident evil グロテスクVer.,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,20,[]


In [17]:
raw_steamspy_data[raw_steamspy_data['languages'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
1398,200350,Starvoid,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
1705,215590,Endless Space QA,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
1877,226760,Rugby Challenge 2,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,1,[]
2438,254270,Dungeonland - All access pass,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
1221,71310,Football Manager 2012 Korean,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,20,[]


In [14]:
#genre
raw_steamspy_data[raw_steamspy_data['genre'].isnull()].sample(5, random_state=0)

,appid,name,developer,publisher,score_rank,positive,negative,userscore,owners,average_forever,average_2weeks,median_forever,median_2weeks,price,initialprice,discount,languages,genre,ccu,tags
1431,201930,Jamestown IGF,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,0,[]
15975,661250,Jarheads,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,NaN,NaN,NaN,NaN,NaN,0,[]
2145,242360,Football Manager 2014 Korean,NaN,NaN,NaN,0,0,0,"0 .. 20,000",0,0,0,0,0.0,0.0,0.0,NaN,NaN,18,[]
20671,783140,Removed,NaN,Slitherine Ltd.,NaN,25,16,0,"200,000 .. 500,000",36,0,22,0,0.0,0.0,0.0,"English, Not supported",NaN,0,"{'Simulation': 1105, 'Action': 35, 'Indie': 33}"
35698,1215390,Animal Revolt Battle Simulator,VDimension,"VDimension, Yodo1 Ltd.",NaN,4151,167,0,"200,000 .. 500,000",499,0,547,0,1499.0,1499.0,0.0,"English, French, Italian, German, Spanish - Sp...",NaN,242,"{'Indie': 384, 'Sandbox': 365, 'Dinosaurs': 33..."


In [23]:
#checking number of actual values in column
raw_steamspy_data['positive'].value_counts().head()

1    4076
2    3265
3    2709
4    2480
0    2224
Name: positive, dtype: int64

In [24]:
raw_steamspy_data['negative'].value_counts().head()

0    10018
1     7241
2     4823
3     3422
4     2623
Name: negative, dtype: int64

In [25]:
raw_steamspy_data['userscore'].value_counts().head()

0      61121
100        4
95         3
80         2
94         2
Name: userscore, dtype: int64

In [27]:
raw_steamspy_data['average_forever'].value_counts().head()

0    47826
1      315
2      112
4      102
3       99
Name: average_forever, dtype: int64

In [28]:
raw_steamspy_data['average_2weeks'].value_counts().head()

0    59741
1       84
2       26
4       23
3       19
Name: average_2weeks, dtype: int64

In [29]:
raw_steamspy_data['median_forever'].value_counts().head()

0    47826
1      314
2      103
4      100
3       99
Name: median_forever, dtype: int64

In [30]:
raw_steamspy_data['median_2weeks'].value_counts().head()

0    59741
1       83
2       25
4       24
3       19
Name: median_2weeks, dtype: int64

In [31]:
raw_steamspy_data['ccu'].value_counts().head()

0    44496
1     5579
2     2002
3     1197
4      798
Name: ccu, dtype: int64

### Removing columns

`score_rank` has too many missing values, `userscore` most values are 0, `average_2weeks`, `median_2weeks` (average and median playtime in the last 2 weeks) and `ccu`(peak conccurrent users yesterday) are time specific and there's no historic data for them in dataset

In [35]:
steamspy_data = raw_steamspy_data.drop(columns=['score_rank', 'userscore', 'average_2weeks', 'median_2weeks', 'ccu'])

In [36]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,owners,average_forever,median_forever,price,initialprice,discount,languages,genre,tags
0,60,Ricochet,Valve,Valve,4218,929,"5,000,000 .. 10,000,000",24,3,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110..."
1,20,Team Fortress Classic,Valve,Valve,6357,979,"5,000,000 .. 10,000,000",136,20,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271..."
2,10,Counter-Strike,Valve,Valve,214651,5470,"10,000,000 .. 20,000,000",7720,232,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3..."
3,70,Half-Life,Valve,Valve,85453,3070,"2,000,000 .. 5,000,000",391,168,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ..."
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,"5,000,000 .. 10,000,000",617,71,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ..."


### Cleaning columns and rows

#### Owners column

In [41]:
steamspy_data['owners'].head()

0     5,000,000 .. 10,000,000
1     5,000,000 .. 10,000,000
2    10,000,000 .. 20,000,000
3      2,000,000 .. 5,000,000
4     5,000,000 .. 10,000,000
Name: owners, dtype: object

In [42]:
# Splitting the owners column into lower and upper bound of an estimation for the amount of owners for the game
steamspy_data[['owners_lowb', 'owners_uppb']] = steamspy_data['owners'].str.replace(',', '').str.split(' .. ', expand=True)

In [44]:
#drop original owners column
steamspy_data = steamspy_data.drop(columns=['owners'])

In [45]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,average_forever,median_forever,price,initialprice,discount,languages,genre,tags,owners_lowb,owners_uppb
0,60,Ricochet,Valve,Valve,4218,929,24,3,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110...",5000000,10000000
1,20,Team Fortress Classic,Valve,Valve,6357,979,136,20,499.0,499.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271...",5000000,10000000
2,10,Counter-Strike,Valve,Valve,214651,5470,7720,232,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3...",10000000,20000000
3,70,Half-Life,Valve,Valve,85453,3070,391,168,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ...",2000000,5000000
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,617,71,999.0,999.0,0.0,"English, French, German, Italian, Spanish - Sp...",Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ...",5000000,10000000


#### Removing rows with missing values

In [56]:
steamspy_data.shape

(61164, 16)

In [57]:
#drop all raws with null values
steamspy_data.dropna(inplace=True)

In [58]:
steamspy_data.shape

(60792, 16)

In [59]:
steamspy_data.isnull().sum()

appid              0
name               0
developer          0
publisher          0
positive           0
negative           0
average_forever    0
median_forever     0
price              0
initialprice       0
discount           0
languages          0
genre              0
tags               0
owners_lowb        0
owners_uppb        0
dtype: int64

In [63]:
#drop rows where `name` is 'none'
steamspy_data = steamspy_data[steamspy_data['name'] != 'none']
steamspy_data.shape

(60790, 16)

#### Languages column

In [73]:
#checking unique languages in the `languages` column
df_languages = steamspy_data['languages'].str.split(', ').explode().unique()
print(df_languages)

['English' 'French' 'German' 'Italian' 'Spanish - Spain' 'Korean'
 'Russian' 'Simplified Chinese' 'Traditional Chinese' 'Dutch' 'Danish'
 'Finnish' 'Japanese' 'Norwegian' 'Polish' 'Portuguese - Portugal'
 'Swedish' 'Thai' 'Turkish' 'Bulgarian' 'Czech' 'Greek' 'Hungarian'
 'Portuguese - Brazil' 'Romanian' 'Spanish - Latin America' 'Ukrainian'
 'Vietnamese' '#lang_slovakian' 'Not supported'
 'Russian\r\nEnglish\r\nSpanish - Spain\r\nFrench\r\nJapanese\r\nCzech'
 '(all with full audio support)' 'Traditional Chinese (text only)'
 '#lang_german;' 'Spanish' '#lang_#lang_spanish*#lang_full_audio'
 '#lang_english (full audio)' 'Arabic' 'Persian'
 '#lang_#lang_russian*#lang_full_audio' 'Filipino' 'Indonesian' 'Basque'
 'Belarusian' 'Welsh' 'Catalan' 'Slovak' 'Latvian' 'Lithuanian'
 'Icelandic' 'Portuguese' 'Croatian' 'Georgian' 'Estonian' 'Slovenian'
 'Albanian' 'Hebrew' 'Hindi' 'Macedonian' 'Kazakh' 'Malay' 'Galician'
 'Serbian' 'Bangla' 'Punjabi (Gurmukhi)' 'Telugu' 'Luxembourgish'
 'Azerbaij

In [120]:
# Define language mapping and replace language names with proper ones
language_mapping = {
    '#lang_slovakian': 'Slovakian',
    '#lang_russian': 'Russian',
    '#lang_german;': 'German',
    '#lang_#lang_spanish*#lang_full_audio': 'Spanish',
    '#lang_english (full audio)': 'English',
    '#lang_russian*#lang_full_audio': 'Russian',
    'Spanish - Spain': 'Spanish',
    'Spanish - Latin America': 'Spanish',
    'Portuguese - Portugal': 'Portuguese',
    'Portuguese - Brazil': 'Portuguese',
}

# Replace language names within each cell
steamspy_data['languages'] = steamspy_data['languages'].str.replace('|'.join(language_mapping.keys()), lambda x: language_mapping[x.group()])


/var/folders/8c/2m03w30x1qd56mkg4zsspzfw0000gp/T/ipykernel_27511/892677419.py:16: FutureWarning: The default value of regex will change from True to False in a future version.
  steamspy_data['languages'] = steamspy_data['languages'].str.replace('|'.join(language_mapping.keys()), lambda x: language_mapping[x.group()])


In [123]:
#checking unique languages again in the `languages` column
df_languages = steamspy_data['languages'].str.split(', ').explode().unique()
print(df_languages)

['English' 'French' 'German' 'Italian' 'Spanish' 'Korean' 'Russian'
 'Simplified Chinese' 'Traditional Chinese' 'Dutch' 'Danish' 'Finnish'
 'Japanese' 'Norwegian' 'Polish' 'Portuguese' 'Swedish' 'Thai' 'Turkish'
 'Bulgarian' 'Czech' 'Greek' 'Hungarian' 'Romanian' 'Ukrainian'
 'Vietnamese' 'Slovakian' 'Not supported'
 'Russian\r\nEnglish\r\nSpanish\r\nFrench\r\nJapanese\r\nCzech'
 '(all with full audio support)' 'Traditional Chinese (text only)'
 '#lang_#lang_spanish*#lang_full_audio' '#lang_english (full audio)'
 'Arabic' 'Persian' '#lang_Russian*#lang_full_audio' 'Filipino'
 'Indonesian' 'Basque' 'Belarusian' 'Welsh' 'Catalan' 'Slovak' 'Latvian'
 'Lithuanian' 'Icelandic' 'Croatian' 'Georgian' 'Estonian' 'Slovenian'
 'Albanian' 'Hebrew' 'Hindi' 'Macedonian' 'Kazakh' 'Malay' 'Galician'
 'Serbian' 'Bangla' 'Punjabi (Gurmukhi)' 'Telugu' 'Luxembourgish'
 'Azerbaijani' 'Afrikaans' 'Dari' 'Mongolian' 'Urdu' 'Irish' 'Swahili'
 'Kannada' 'Tamil' 'Maori' 'Valencian' 'Bosnian' 'Scots' 'Gujarati'

In [124]:
# For some reason, some of the values weren't getting mapped, so they will be droped once each language gets it's own column

# Split languages into separate columns
language_dummies = steamspy_data['languages'].str.get_dummies(', ')

# Display the updated DataFrame
steamspy_data = pd.concat([steamspy_data, language_dummies], axis=1)
steamspy_data.shape

(60790, 98)

In [ ]:
#check column names to see which ones need to be dropped
column_names = list(steamspy_data.columns.values)
print(column_names)

In [ ]:
#drop columns with wrong names
steamspy_data = steamspy_data.drop(columns=['#lang_#lang_spanish*#lang_full_audio', 
                                            '#lang_Russian*#lang_full_audio', 
                                            '#lang_english (full audio)', 
                                            '(all with full audio support)', 
                                            'Not supported', 
                                            "Russian\r\nEnglish\r\nSpanish\r\nFrench\r\nJapanese\r\nCzech", 
                                            'Traditional Chinese (text only)']
                                            )

In [141]:
column_names = list(steamspy_data.columns.values)
print(column_names)

['appid', 'name', 'developer', 'publisher', 'positive', 'negative', 'average_forever', 'median_forever', 'price', 'initialprice', 'discount', 'languages', 'genre', 'tags', 'owners_lowb', 'owners_uppb', 'Afrikaans', 'Albanian', 'Arabic', 'Azerbaijani', 'Bangla', 'Basque', 'Belarusian', 'Bosnian', 'Bulgarian', 'Catalan', 'Croatian', 'Czech', 'Danish', 'Dari', 'Dutch', 'English', 'Estonian', 'Filipino', 'Finnish', 'French', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Hebrew', 'Hindi', 'Hungarian', 'Icelandic', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malay', 'Maori', 'Marathi', 'Mongolian', 'Nepali', 'Norwegian', 'Persian', 'Polish', 'Portuguese', 'Punjabi (Gurmukhi)', 'Romanian', 'Russian', 'Scots', 'Serbian', 'Simplified Chinese', 'Slovak', 'Slovakian', 'Slovenian', 'Spanish', 'Swahili', 'Swedish', 'Tamil', 'Tatar', 'Telugu', 'Thai', 'Traditional Chinese', 'Turkish', 'Ukrain

In [142]:
# Specify the columns to sum
column_sums = steamspy_data.loc[:, 'Afrikaans':'Zulu'].sum()

# Calculate the sum for the specified columns
column_sums.sort_values(ascending=False)

English                60449
German                 15475
French                 14860
Simplified Chinese     14609
Spanish                14364
Russian                14127
Japanese               12004
Italian                10331
Portuguese              9584
Korean                  7616
Traditional Chinese     7219
Polish                  5955
Turkish                 4139
Dutch                   3410
Czech                   2578
Swedish                 2280
Ukrainian               2258
Hungarian               2114
Arabic                  2036
Thai                    1955
Danish                  1914
Norwegian               1821
Finnish                 1818
Romanian                1620
Greek                   1618
Bulgarian               1416
Vietnamese              1307
Indonesian               113
Slovak                    70
Catalan                   68
Hindi                     58
Hebrew                    40
Malay                     37
Belarusian                37
Serbian       

In [143]:
# Calculate the sum of values in the same row for the specified column range
languages_sum = steamspy_data.loc[:, 'Afrikaans':'Zulu'].sum(axis=1)

# Add the row sums as a new column in the DataFrame
steamspy_data['languages_sum'] = languages_sum

In [149]:
# Since we have a lot of languages, we are going to drop all languages that with sum less 1,000

# Filter the columns based on the sum condition
columns_to_drop = column_sums[column_sums < 1000].index

# Drop the columns from the DataFrame
steamspy_data = steamspy_data.drop(columns=columns_to_drop)

In [151]:
#drop original languages column
steamspy_data = steamspy_data.drop(columns=['languages'])

In [152]:
steamspy_data.head()

,appid,name,developer,publisher,positive,negative,average_forever,median_forever,price,initialprice,discount,genre,tags,owners_lowb,owners_uppb,Arabic,Bulgarian,Czech,Danish,Dutch,English,Finnish,French,German,Greek,Hungarian,Italian,Japanese,Korean,Norwegian,Polish,Portuguese,Romanian,Russian,Simplified Chinese,Spanish,Swedish,Thai,Traditional Chinese,Turkish,Ukrainian,Vietnamese,languages_sum
0,60,Ricochet,Valve,Valve,4218,929,24,3,499.0,499.0,0.0,Action,"{'Action': 588, 'FPS': 132, 'Multiplayer': 110...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
1,20,Team Fortress Classic,Valve,Valve,6357,979,136,20,499.0,499.0,0.0,Action,"{'Action': 757, 'FPS': 318, 'Multiplayer': 271...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,1,0,0,0,9
2,10,Counter-Strike,Valve,Valve,214651,5470,7720,232,999.0,999.0,0.0,Action,"{'Action': 5443, 'FPS': 4857, 'Multiplayer': 3...",10000000,20000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
3,70,Half-Life,Valve,Valve,85453,3070,391,168,999.0,999.0,0.0,Action,"{'FPS': 2306, 'Sci-fi': 1819, ""1990's"": 1816, ...",2000000,5000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8
4,80,Counter-Strike: Condition Zero,Valve,Valve,20911,1995,617,71,999.0,999.0,0.0,Action,"{'Action': 1369, 'FPS': 1027, 'Shooter': 759, ...",5000000,10000000,0,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,0,1,1,0,0,1,0,0,0,8


#### Tags column